# tests

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame({
    "vp": [6,3,6,5,10], 
    "p": [4,4,2,12,23], 
    'n': [5,14,1,32,12],
    'r': [3,7,3,27,5],
    'vr': [2,6,0,18,14],
})

In [ ]:
df

In [ ]:
df.apply(lambda x:x/x.sum(),axis=1)

# Introduction

- outliers regularisation method = mean
- mean regularisation columns = [
    "PPDCategoryType",
    "PropertyType",
    "Duration",
    'County',
    'TownCity'
]
- outlier_deletion = (isLog = False) then (isLog = True)

--> r2 = 0.69 - 0.70

In [1]:
config_processing = {
    # ----- default
    # """
    #     r2 = 0.6827
    #     inputs = [
    #         'OldNew' ,  
    #         'Duration' , 
    #         'PPDCategoryType',
    #         # Month
    #         *[k for k in data.columns if ('Month' in k)],
    #         # Weights
    #         *[k for k in data.columns if ('Weights' in k) and (not ("Transf" in k))],
    #         # # zscore
    #         *[k for k in data.columns if ('WeightsTransf' in k)],
    #         # PropertyType
    #         *[k for k in data.columns if ('PropertyType' in k)],
    #         # target
    #         target
    #     ]
    # """
    "default" : {
        "method" : "mean",
        "groupby" : [
            "PPDCategoryType",
            "PropertyType",
            "Duration",
            'District',
            'TownCity',
        ],
        "isLog" : False,
        'classify_outliers_cycle': 2
    },
    # ----- v1
    # """
    #     r2 = 0.6813    
    #     inputs = [
    #         'OldNew' ,  
    #         'Duration' , 
    #         'PPDCategoryType',
    #         # Month
    #         *[k for k in data.columns if ('Month' in k)],
    #         # Weights
    #         *[k for k in data.columns if ('Weights' in k) and (not ("Transf" in k))],
    #         # PropertyType
    #         *[k for k in data.columns if ('PropertyType' in k)],
    #         # target
    #         target
    #     ]
    # """
    "version 1" : {
        "method" : "mean",
        "groupby" : [
            "PropertyType",
            "Duration",
            'County',
            'District',
            'TownCity',
        ],
        "isLog" : False,
        'classify_outliers_cycle': 2
    },
    # ----- v2 
    # """
    #     r2 = 0.6964    
    #     inputs = [
    #         'OldNew' ,  
    #         'Duration' , 
    #         'PPDCategoryType',
    #         # Month
    #         *[k for k in data.columns if ('Month' in k)],
    #         # Weights
    #         *[k for k in data.columns if ('Weights' in k) and (not ("Transf" in k))],
    #         # PropertyType
    #         *[k for k in data.columns if ('PropertyType' in k)],
    #         # target
    #         target
    #     ]
    # """
    "version 2" : {
        "method" : "mean",
        "groupby" : [
            "PropertyType",
            'County',
            "PostcodeArea",
            'District',
            'TownCity',
        ],
        "isLog" : True,
        'classify_outliers_cycle': 2
    },
    # ----- v3 
    # """
    #     r2 = ??    
    #     inputs = [
    #         'OldNew' ,  
    #         'Duration' , 
    #         'PPDCategoryType',
    #         # Month
    #         *[k for k in data.columns if ('Month' in k)],
    #         # Weights
    #         *[k for k in data.columns if ('Weights' in k) and (not ("Transf" in k))],
    #         # PropertyType
    #         *[k for k in data.columns if ('PropertyType' in k)],
    #         # target
    #         target
    #     ]
    # """
    "version 3" : {
        "method" : "mean",
        "groupby" : [
            "PropertyType",
            "Duration",
            'County',
            "PostcodeArea",
            'District',
            'TownCity',
            
        ],
        "isLog" : False,
        'classify_outliers_cycle': 2
    },
}

In [2]:
choosed_version = "version 2"
# ----- define elements
outliers_regularisation_method = config_processing[choosed_version]["method"]
groupbycolumns = config_processing[choosed_version]["groupby"]
logoutlierfence= config_processing[choosed_version]["isLog"]
classify_outliers_cycle= config_processing[choosed_version]["classify_outliers_cycle"]

# Load data

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import scipy as sp
pd.set_option('float_format', '{:f}'.format)
plt.style.use('ggplot')


In [4]:
from matplotlib import style 

print(plt.style.available)

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [5]:
columns = [
    'Id',
    'Price',
    'TransferDate',
    'Postcode',
    'PropertyType',
    'OldNew',
    'Duration',
    'PAON',
    'SAON',
    'Street',
    'Locality' ,
    'TownCity',
    'District',
    'County',
    'PPDCategoryType',
    'RecordStatus',
]
houses_loaded = pd.read_csv('data/pp-2021.csv',names=columns)

In [ ]:
houses_loaded.head()

# Functions

## visualization

In [6]:
def find_df_name(df):
   name = [name for name, obj in globals().items() if id(obj) == id(df)]
   return name[0] if name else None

In [7]:
def bar_catg_feature(col_name,df_data):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))
    # data
    df_feature = df_data.loc[:,[col_name,"Price"]]

    df_grouped = df_feature.groupby(col_name)["Price"].mean()
    # first plot
    sns.barplot(
        x=df_grouped.index, 
        y=df_grouped.values, 
        ax=axes[0]
    )
    # second plot
    values = df_feature[col_name].value_counts(dropna=False)
    sns.barplot(
        x=values.index, 
        y=values.values,
        ax=axes[1]
    )
    # Add titles
    axes[0].set_title(f'Average Price by {col_name} (data={find_df_name(df_data)})')
    axes[1].set_title(f'distribution of {col_name} (data={find_df_name(df_data)})')
    # show
    plt.tight_layout()
    plt.show()

In [8]:
def bar_catg_pairfeature(colname1,colname2,colname3='Price',df_data=None):
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20, 6))
    # data
    df_feature = df_data.loc[:,[colname1,colname2,colname3]]
    sns.barplot(
        data = df_feature,
        x=colname2, 
        y=colname3,
        hue=colname1,
        ax=axes
    )
    # Add titles
    axes.set_title(f'Average Price by {colname1}, {colname2} (data={find_df_name(df_data)})')
    # show
    plt.tight_layout()
    plt.show()


In [9]:
def bar_geo_feature(colname1,colname2='Price',df_data = None,ascending=False,method='mean'):
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20, 12))
    # data
    if method=='mean':
        df_feature = df_data.groupby(colname1)[colname2].mean().sort_values(ascending=ascending)[:50]
    elif method=='median':
        df_feature = df_data.groupby(colname1)[colname2].median().sort_values(ascending=ascending)[:50]
    sns.barplot(
        x=df_feature.index, 
        y=df_feature.values,
        ax=axes
    )
    # Add titles
    axes.tick_params(axis='x', rotation=90)
    axes.set_title(f'{method} of {colname2} by {colname1} (data={find_df_name(df_data)})')
    # show
    # plt.tight_layout()
    plt.show()

In [10]:
def box_geo_feature(colname1,colname2='Price',df_data = None,method='mean'):
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 6))
    if method=='mean':
        df_grouped_mean = df_data.groupby(colname1)[colname2].mean().sort_values(ascending=False)
    elif method=='median':
        df_grouped_mean = df_data.groupby(colname1)[colname2].median().sort_values(ascending=False)
    sns.boxplot(df_grouped_mean)
    plt.title(f'Distribution of {method} of {colname2} by {colname1} (data={find_df_name(df_data)})')
    plt.show()

In [11]:
def price_distribution(df,bins=100,kind='bar'):
    df['Price'].value_counts(bins=bins).sort_index().plot(kind=kind,figsize=(20,16))
    # show
    plt.tight_layout()
    plt.show()

In [12]:
def price_boxplot(df,col_name=None,isLog=False):
    kwargs = {
        'y' : np.log(df['Price']) if isLog else df['Price']
    }
    if col_name: kwargs['x'] = df[col_name]  
    ax = sns.boxplot(**kwargs)
    title = f'Price by {col_name}' if col_name!= None else 'Price' 
    ax.set_title(title + f' :(log={isLog},data={find_df_name(df)})',fontsize=10)
    plt.show()

In [13]:
def price_scatterplot(df,col_name=None,isColDivided=False,isLog=False):
    
    if isColDivided:
        # -- inputs
        vv = df[col_name].value_counts().index
        ncols = 2
        nrows = len(vv)//ncols + len(vv)%ncols
        # -- canvas
        fig, ax = plt.subplots(
            nrows = nrows,
            ncols = ncols, 
            figsize=(10*ncols,8*nrows)
        )
        # -- draw
        i=j=0
        for ptype in vv:
            dd = df.loc[df[col_name] == ptype,['Price',col_name]]
            if nrows == 1:
                sns.scatterplot(
                    x = dd[col_name],
                    y = np.log(dd.Price) if isLog else dd.Price,
                    ax = ax[j]
                ).set(title=f'{col_name} ({ptype})')
                # ax[j].plot(dd)
                # ax[j].title.set_text(f'{col_name}({ptype})')
            else : 
                sns.scatterplot(
                    x = dd[col_name],
                    y = np.log(dd.Price) if isLog else dd.Price,
                    ax=ax[i, j]
                ).set(title=f'{col_name} ({ptype})')
                # ax[i, j].plot(dd)
                # ax[i, j].title.set_text(f'{col_name}({ptype})')
            if j == 1:
                i += 1
                j=0
            else: j+=1
        fig.suptitle(f'Price by {col_name} :(log={isLog},data={find_df_name(df)})')
    else:
        kwargs = {
            'y' : np.log(df['Price']) if isLog else df['Price']
        }
        if col_name: kwargs['x'] = df[col_name] 
        ax = sns.scatterplot(**kwargs)
        title = f'Price by {col_name}' if col_name!= None else 'Price' 
        ax.set_title(title + f' :(log={isLog},data={find_df_name(df)})',fontsize=10)
    plt.show()

In [14]:
def visualise_grps_bycol(df,col_name,ncols=2):
    vv = df[col_name].value_counts().index
    nrows = len(vv)//ncols + len(vv)%ncols
    fig , ax = plt.subplots(
        nrows = nrows,
        ncols = ncols,
        figsize=(10*ncols,8*nrows)
    )
    i = j = 0
    for ptype in vv:
        print(f"{ptype} : ({i},{j})")
        dd = df.loc[df[col_name] == ptype,'Price']
        if nrows == 1:
            ax[j].plot(dd)
            ax[j].title.set_text(f'{col_name}({ptype})')
        else : 
            ax[i, j].plot(dd)
            ax[i, j].title.set_text(f'{col_name}({ptype})')
        if j == 1:
            i += 1
            j=0
        else: j+=1
    plt.show()

In [15]:
def visualise_grps(df,col_name,ncols=2):
    bar_catg_feature(col_name,df_data = df)
    box_geo_feature(col_name,df_data = df)
    price_boxplot(df,col_name=col_name,isLog=False)
    price_boxplot(df,col_name=col_name,isLog=True)
    visualise_grps_bycol(df,col_name,ncols=ncols)

In [16]:
def visualise_global(df,ncols=2,bins=100,grp_columns = [],geo_col = [],scatter_col=[]):
    """
        grp_columns = ["PropertyType","Duration","OldNew"]
        geo_col = ["County","District","TownCity"]
    """
    # -- Price
    print("###################################################### Price ######################################################")
    price_distribution(df,bins=bins,kind='bar')
    price_boxplot(df,col_name=None,isLog=False)
    price_boxplot(df,col_name=None,isLog=True)
    
    # -- grp_columns
    for col in grp_columns:
        print(f"###################################################### {col} ######################################################")
        visualise_grps(df,col,ncols)
    
    # -- geo_col
    for col in geo_col:
        print(f"###################################################### {col} ######################################################")
        box_geo_feature(col,df_data = df)
        bar_geo_feature(col,colname2='Price',df_data = df,ascending=False,method='mean')
    
    # -- scatterplot
    for col in scatter_col:
        print(f"###################################################### {col} ######################################################")
        price_scatterplot(df,col_name=col,isColDivided=False,isLog=False)

## aggs & transformation

In [17]:
def fix_dfHouses(df):
    # PropertyType : D = Detached, S = Semi-Detached, T = Terraced, F = Flats/Maisonettes, O = Other
    df.loc[df['PropertyType']=='T','PropertyType'] = 'Terraced'
    df.loc[df['PropertyType']=='S','PropertyType'] = 'SemiDetached'
    df.loc[df['PropertyType']=='D','PropertyType'] = 'Detached'
    df.loc[df['PropertyType']=='F','PropertyType'] = 'Flat'
    df.loc[df['PropertyType']=='O','PropertyType'] = 'Other'
    # OldNew : Y = New, N = Old
    df.loc[df['OldNew']=='Y','OldNew'] = 'New'
    df.loc[df['OldNew']=='N','OldNew'] = 'Old'
    # Duration : F = Freehold, L= Leasehold
    df.loc[df['Duration']=='F','Duration'] = 'Freehold'
    df.loc[df['Duration']=='L','Duration'] = 'Leasehold'
    # PPDCategoryType : A = StandardPrice (Standard Price Paid entry) ,  B = AdditionalPrice (Additional Price Paid)
    df.loc[df['PPDCategoryType']=='A','PPDCategoryType'] = 'StandardPrice'
    df.loc[df['PPDCategoryType']=='B','PPDCategoryType'] = 'AdditionalPrice'
    # TransferDate --> Day, Month
    df['TransferDate'] = pd.to_datetime(df['TransferDate'])
    df['TransferMonth'] = df['TransferDate'].dt.month
    df['TransferDay'] = df['TransferDate'].dt.day
    return df

In [18]:
def categorise(col,df_data,weight_func = lambda x:x):
    cat_col = df_data.groupby(col)['Price'].mean()
    # Calculate the quartiles:
    q25 = np.percentile(cat_col, 25)
    q75 = np.percentile(cat_col, 75)
    max_q = min(cat_col.max(),q75+1.5*(q75-q25))
    min_q = max(cat_col.min(),q25-1.5*(q75-q25))
    # caterg
    def col_cat(x,min_q,q25,q75,max_q):
        if x["Price"] < min_q : return "Very Poor"
        elif (x["Price"] >= min_q) and (x["Price"] <= q25) : return "Poor"
        elif (x["Price"] > q25) and (x["Price"] <= q75) : return "Normal"
        elif(x["Price"] > q75) and (x["Price"] <= max_q) : return "Rich"
        else: return "Very Rich"
    def coef_calc(x,coltype,coef):
        return coef_types[x[coltype]]*x['counts']
    # def cat_dfdata(df,col,weights):
    #     df_cp = df.copy()
    #     df_cp.loc[:,col+'Weights'] = df_cp.apply(lambda x:weights[x[col]],axis=1)
    #     return df_cp
    # caterg
    coltype = col+'Type'
    houses_categorised = pd.DataFrame(df_data.apply(lambda x: col_cat(x,min_q=min_q,q25=q25,q75=q75,max_q=max_q),axis=1),columns = [coltype])
    houses_categorised = pd.concat([df_data,houses_categorised], axis=1)
    houses_categorised_byType = houses_categorised.loc[:,[col,coltype]].groupby([col,coltype]).agg(
        counts = pd.NamedAgg(column=coltype, aggfunc=lambda x:x.count()),
    )
    coef_types_weights = {
        'Normal' : 0,
        'Poor': -2,
        'Rich': 2,
        'Very Poor': -4,
        'Very Rich': 4,
    }
    coef_types = {k:weight_func(v) for k,v in coef_types_weights.items()}
    houses_categorised_byType = houses_categorised_byType.groupby(level=[col],group_keys=False).apply(lambda x:x/x.sum()).reset_index()
    houses_categorised_byType['weights'] = houses_categorised_byType.apply(lambda x: coef_calc(x,coltype,coef_types),axis=1)
    categories_weights = houses_categorised_byType.groupby(col)['weights'].sum()
    
    return {
        'categories_weights':categories_weights,
        col+'Weights': df_data.apply(lambda x:categories_weights[x[col]],axis=1)
    }

In [19]:
def classify_outliers(df_data,colname='Price',by_log=False):
    # target
    if not by_log:
        target_col = df_data[colname]
        target_metrics = target_col.describe()
    else:
        target_col = np.log(df_data[colname])
        target_metrics = target_col.describe()
    # metrics 
    price_q25= target_metrics["25%"]
    price_q50= target_metrics["50%"]
    price_q75= target_metrics["75%"]
    price_IQR = price_q75 - price_q25
    price_max= min(target_metrics["max"],price_q75+1.5*price_IQR)
    price_min= max(target_metrics["min"],price_q25-1.5*price_IQR)
    mask = np.logical_and((target_col <= price_max), (target_col >= price_min))
    normal_PriceWise_df = df_data.loc[mask,:]
    expensive_PriceWise_df = df_data.loc[~mask,:]
    return {
        "q25":price_q25,
        "q50":price_q50,
        "q75":price_q75,
        "IQR":price_IQR,
        "max":price_max,
        "min":price_min,
        "normal_PriceWise_df":normal_PriceWise_df,
        "expensive_PriceWise_df":expensive_PriceWise_df
    }

In [20]:
aa = pd.DataFrame()
aa.empty

True

In [21]:
def regulate_outliers(df_data, method='mean', groupby=None,bothsides=False,maxmin=False,df_reference=pd.DataFrame()):
    """
        1- expl:
            maxmin: 
                - x>outliser_bar_max ---> x=outliser_bar_max
                - x<outliser_bar_min ---> x=outliser_bar_min
            bothsides:
                - (x>outliser_bar_max) or (x<outliser_bar_min) ---> x=groupby(method)
            groupby:
                - name of column of group by
                - list or str
            
        2- Notes:
            - "maxmin" is prioritised over "bothsides" 
    """
    data = df_data.copy()
    df_data_reference = df_reference.copy() if not df_reference.empty else data
    if groupby != None:
        df_data_grps = df_data_reference.groupby(groupby).agg(
            min = pd.NamedAgg(column='Price',aggfunc='min'),
            outlier_bar_Min = pd.NamedAgg(column='Price',aggfunc=lambda x:max(min(x),np.percentile(x,25) - 1.5*(np.percentile(x,75)-np.percentile(x,25)))),
            q25 = pd.NamedAgg(column='Price',aggfunc=lambda x:np.percentile(x,25)),
            q50 = pd.NamedAgg(column='Price',aggfunc=lambda x:np.percentile(x,50)),
            q75 = pd.NamedAgg(column='Price',aggfunc=lambda x:np.percentile(x,75)),
            outlier_bar_Max = pd.NamedAgg(column='Price',aggfunc=lambda x:min(max(x),np.percentile(x,75) + 1.5*(np.percentile(x,75)-np.percentile(x,25)))),
            max = pd.NamedAgg(column='Price',aggfunc='max'),
            median = pd.NamedAgg(column='Price',aggfunc='median'),
            mean = pd.NamedAgg(column='Price',aggfunc='mean'),
        )
        def fix_outliers(x,df_data_grps,method,groupby,bothsides):
            # print(x)
            if not (tuple(x[groupby]) in df_data_grps.index): return np.nan 
            s_x = df_data_grps.loc[tuple(x[groupby]),:]
            if x['Price']<s_x['outlier_bar_Min']: 
                if maxmin or (not bothsides): return s_x['outlier_bar_Min'] 
                else: return s_x[method]
            elif x['Price']>s_x['outlier_bar_Max']: 
                if maxmin: return s_x['outlier_bar_Max'] 
                else: return s_x[method]
            else : return x['Price']
        kwargs = {
            'df_data_grps':df_data_grps,
            'method':method,
            'groupby':groupby,
            'bothsides':bothsides,
        } 
    else:
        metric_method = df_data_reference.Price.agg(method)
        df_data_grps = df_data_reference.Price.describe()
        
        def fix_outliers(x,df_data_grps,metric_method,bothsides):
            # -- metrics
            min_p = df_data_grps['min']
            max_p = df_data_grps['max']
            q25_p = df_data_grps['25%']
            q75_p = df_data_grps['75%']
            bar_max = min(max_p,q75_p + 1.5*(q75_p-q25_p))
            bar_min = max(min_p,q25_p - 1.5*(q75_p-q25_p))
            # -- regulate
            if x['Price']<bar_min:
                if maxmin or (not bothsides): return bar_min 
                else: return metric_method
            elif x['Price']>bar_max: 
                if maxmin: return bar_max 
                else: return metric_method
            else : return x['Price']
        kwargs = {
            'df_data_grps':df_data_grps,
            'metric_method':metric_method,
            'bothsides':bothsides
        }
    return data.apply(lambda x:fix_outliers(x,**kwargs),axis=1)
        

In [22]:
def stratified_sampling(df,colname,frac,random_state=None):
    sampling_options = {
        "frac":frac
    }
    if random_state!= None : sampling_options['random_state']=random_state
    return df.groupby(colname, group_keys=False).apply(lambda x: x.sample(**sampling_options))

In [23]:
def transform_feat(df,cols=[],func_transform=lambda x:x):
    df_cp = df.copy()
    for col in cols:
        df_cp[col+"Transf"] = df_cp[col].apply(func_transform)
    return df_cp

# Discovery

## Exploration

In [ ]:
houses_loaded.info()

In [ ]:
houses_loaded.describe()

## Cleaning

In [ ]:
cat_features = [
    'PropertyType',
    'OldNew',
    'Duration',
    'TownCity',
    'District',
    'County',
    'PPDCategoryType',
    'RecordStatus',
]
for col in cat_features:
    print("Number of categories of Feature '{0}' is : {1} ".format(col, len(houses_loaded[col].unique())))
    print(houses_loaded[col].value_counts(dropna=False).index.tolist())
    print('--------------------------------------------------------------------------------------------')

In [24]:
houses_loaded = fix_dfHouses(houses_loaded)

In [ ]:
houses_loaded.head()

## Explore outliers

In [ ]:
housesLoaded_metrics = houses_loaded.Price.describe()
max_fence = housesLoaded_metrics['75%'] + 1.5*(housesLoaded_metrics['75%']-housesLoaded_metrics['25%'])
min_fence = housesLoaded_metrics['25%'] - 1.5*(housesLoaded_metrics['75%']-housesLoaded_metrics['25%'])
min_fence,max_fence

In [ ]:
outliers_houses = houses_loaded.loc[(houses_loaded.Price>max_fence) | (houses_loaded.Price<min_fence),:].reset_index()

In [ ]:
mask = (outliers_houses.OldNew == "Old") & (outliers_houses.TownCity == "LONDON") & (outliers_houses.PPDCategoryType == "AdditionalPrice")
hyper_exp = outliers_houses.loc[mask,:]
reas_exp = outliers_houses.loc[~mask,:]

In [ ]:
visualise_global(reas_exp)

In [ ]:
price_scatterplot(outliers_houses,col_name='OldNew',isColDivided=False,isLog=False)

## Visualisation

In [ ]:
visualise_global(
    houses_loaded,
    grp_columns = ["PropertyType","Duration","OldNew"],
    geo_col = ["County","District","TownCity"]
)

# Preprocessing

### PostcodeArea

In [25]:
houses_loaded['PostcodeArea'] = houses_loaded.Postcode.str.extract(r'([a-z]+)',flags=re.I)

In [26]:
mask_pc = houses_loaded.PostcodeArea.isna()
town_city_PostCodesArea = houses_loaded.groupby('TownCity')["PostcodeArea"].apply(lambda x:pd.Series(list(set(x))).dropna().tolist()[0])
houses_loaded.loc[mask_pc,'PostcodeArea']= houses_loaded.loc[(mask_pc),['PostcodeArea',"TownCity"]].apply(lambda x:town_city_PostCodesArea[x["TownCity"]],axis=1)

In [ ]:
houses_loaded.PostcodeArea.value_counts(dropna=True)

### normal_PriceWise_houses && expensive_PriceWise_houses

- 'logoutlierfence' is declared in "introduction" section

#### iteratif

In [ ]:
pre_classified_houses = classify_outliers(
    df_data=houses_loaded,
    colname='Price',
    by_log = logoutlierfence
)
pre_expensive_PriceWise_houses = pre_classified_houses["expensive_PriceWise_df"]
pre_normal_PriceWise_houses = pre_classified_houses["normal_PriceWise_df"]

In [ ]:
classified_houses = classify_outliers(
    df_data=pre_normal_PriceWise_houses,
    colname='Price',
    by_log = not logoutlierfence
)
expensive_PriceWise_houses = classified_houses["expensive_PriceWise_df"]
normal_PriceWise_houses = classified_houses["normal_PriceWise_df"]

#### recursif

In [27]:
normal_PriceWise_houses = houses_loaded.copy()
for i in range(classify_outliers_cycle):
    classified_houses = classify_outliers(
        df_data=normal_PriceWise_houses,
        colname='Price',
        by_log = logoutlierfence
    )
    expensive_PriceWise_houses = classified_houses["expensive_PriceWise_df"]
    normal_PriceWise_houses = classified_houses["normal_PriceWise_df"]
    logoutlierfence = not logoutlierfence

#### Visualization

In [ ]:
grp_columns = ["PropertyType","Duration","OldNew"]
geo_col = ["County","District","TownCity","PostcodeArea"]

visualise_global(
    normal_PriceWise_houses,
    bins=100,
    grp_columns=grp_columns,
    geo_col=geo_col
)

In [ ]:
visualise_global(
    expensive_PriceWise_houses,
    grp_columns = ["PropertyType","Duration","OldNew"],
    geo_col = ["County","District","TownCity"],
    bins=100
)

### houses_regulated

- 'outliers_regularisation_method', 'groupbycolumns' are declared in "introduction" section

#### option 1: Regulate "expensive_PriceWise_houses" and merge it with "normal_PriceWise_houses"

In [28]:
reg_expensive_PriceWise_houses = expensive_PriceWise_houses.copy()
reg_expensive_PriceWise_houses['Price'] = regulate_outliers(
    df_data = expensive_PriceWise_houses, 
    method=outliers_regularisation_method, 
    groupby = groupbycolumns,
    bothsides=False,
    maxmin=False,
    df_reference=normal_PriceWise_houses
)

In [29]:
houses_regulated = pd.concat([normal_PriceWise_houses,reg_expensive_PriceWise_houses], axis=0)

In [30]:
houses_regulated = houses_regulated.loc[houses_regulated.Price.notna(),:]

In [31]:
houses_regulated.reset_index(drop=True,inplace=True)

In [32]:
houses_regulated['Price'] = regulate_outliers(
    df_data = houses_regulated, 
    method=outliers_regularisation_method, 
    groupby = groupbycolumns,
    bothsides=False,
    maxmin=False
)

#### option 2: Regulate "normal_PriceWise_houses"

In [ ]:
houses_regulated = normal_PriceWise_houses.copy()
houses_regulated['Price'] = regulate_outliers(
    df_data = normal_PriceWise_houses, 
    method=outliers_regularisation_method, 
    groupby = groupbycolumns,
    bothsides=False,
    maxmin=False
)

#### Visualization

In [ ]:
houses_regulated.describe()

In [ ]:
visualise_global(
    houses_regulated,
    grp_columns = ["PropertyType","Duration","OldNew"],
    geo_col = ["County","District","TownCity"]
)

### Houses

In [ ]:
houses = houses_regulated.copy()
houses.reset_index(drop=True,inplace=True)

In [ ]:
houses.isna().sum()

#### "Weights" Feature

In [ ]:
weight_func = lambda x:x

##### County

In [ ]:
categorised_counties_data = categorise(
    col="County",
    df_data=houses,
    weight_func = weight_func
)
houses.loc[:,'CountyWeights'] = categorised_counties_data['CountyWeights']
counties_weights = categorised_counties_data['categories_weights']

##### District

In [ ]:
categorised_districts_data = categorise(
    col="District",
    df_data=houses,
    weight_func = weight_func
)
houses.loc[:,'DistrictWeights'] = categorised_districts_data['DistrictWeights']
districts_weights = categorised_districts_data['categories_weights']

##### TownCity

In [ ]:
categorised_cities_data = categorise(
    col="TownCity",
    df_data=houses,
    weight_func = weight_func
)
houses.loc[:,'TownCityWeights'] = categorised_cities_data['TownCityWeights']
cities_weights = categorised_cities_data['categories_weights']

##### Postcode

In [ ]:
categorised_PostcodeArea_data = categorise(
    col="PostcodeArea",
    df_data=houses,
    weight_func = weight_func
)
houses.loc[:,'PostcodeAreaWeights'] = categorised_PostcodeArea_data['PostcodeAreaWeights']
PostcodeArea_weights = categorised_PostcodeArea_data['categories_weights']

##### Visualisation

In [ ]:
visualise_global(
    houses,
    bins=100,
    scatter_col=[
        "CountyWeights",
        "DistrictWeights",
        "TownCityWeights",
        "PostcodeAreaWeights"
    ]
)

#### Feature Transformation

In [ ]:
houses_featured = houses.copy()
col_scatter = []

##### option 1: ('transformation_func' for each column)

In [ ]:
scatter_col = {
    "CountyWeights":lambda x: x**3,
    "DistrictWeights":lambda x: x**3,
    "TownCityWeights":lambda x: x**3,
    "PostcodeAreaWeights":lambda x: x**3
}
for k,v in scatter_col.items():
    houses_featured = transform_feat(
        df=houses_featured,
        cols=[k],
        func_transform=v
    )

##### option 2: ('transformation_func' for all)

In [ ]:
# sigmoid
transformation_func = lambda x: sp.special.expit(x)

In [ ]:
scatter_col = [
    "CountyWeights",
    "DistrictWeights",
    "TownCityWeights",
    "PostcodeAreaWeights"
]
houses = transform_feat(
    df=houses_featured,
    cols=scatter_col,
    func_transform=transformation_func
)

##### Visualization

In [ ]:
houses_featured

In [ ]:
col_scatter_list = scatter_col if type(scatter_col)==list else scatter_col.keys()
col_scatter = [col+'Transf' for col in col_scatter_list]
visualise_global(
    houses_featured,
    bins=100,
    scatter_col=col_scatter
)

## Data Encoding

### houses_selected

In [ ]:
selected_features = [
    'Price',
    "PropertyType",
    "OldNew",
    "Duration",
    "PPDCategoryType",
    "TransferMonth",
    "TransferDay",
    "CountyWeights",
    "DistrictWeights",
    "TownCityWeights",
    "PostcodeAreaWeights",
    *col_scatter
]
houses_selected = houses_featured.loc[:,selected_features].copy()
houses_selected.reset_index(drop=True,inplace=True)

In [ ]:
houses_selected

### houses_encoded

In [ ]:
houses_encoded = houses_selected.copy()

#### Label encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
houses_encoded['PPDCategoryType'] = label_encoder.fit_transform(houses_encoded['PPDCategoryType'])
houses_encoded['OldNew'] = label_encoder.fit_transform(houses_encoded['OldNew'])
houses_encoded['Duration'] = label_encoder.fit_transform(houses_encoded['Duration'])

#### OneHotEncoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder
categ_features = [
    "PropertyType",
    "TransferMonth"
]

one_hot_encoder = OneHotEncoder(sparse_output=False) # feature_name_combiner=lambda x,y:str(x)+"_"+str(y)
encoded_categ_features = one_hot_encoder.fit_transform(houses_encoded[categ_features])

df_encoded_categ_features = pd.DataFrame(
    encoded_categ_features, 
    columns = one_hot_encoder.get_feature_names_out(categ_features)
)

In [ ]:
houses_encoded = pd.concat([houses_encoded,df_encoded_categ_features], axis=1)
houses_encoded

#### fixing "houses_encoded"

In [ ]:
houses_encoded.drop(
    [
        "PropertyType",
        "TransferMonth",
        "TransferDay"
    ],
    axis=1,
    inplace=True
)

In [ ]:
input_features_encoded = [k for k in houses_encoded.columns if (k != "Price") and not ('Weights' in k)]
houses_encoded = houses_encoded.astype(
    {
        k:int for k in input_features_encoded
    }
)

#### show "houses_encoded"

In [ ]:
houses_encoded.info()

In [ ]:
houses_encoded.corr()['Price'].head(50)

In [ ]:
plt.figure(figsize = (25,25))
sns.heatmap(houses_encoded.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.show()

# Model Selection

- for Model directory for saving and loading is stored in "model_dir_location"

In [ ]:
model_dir_location = f'./models/{outliers_regularisation_method}/'

## Data (for Modeling) & Data_Unseen (for test)

In [ ]:
data = stratified_sampling(
    df=houses_encoded,
    colname="PPDCategoryType",
    frac=0.95,
    random_state=1
)
data_unseen = houses_encoded.drop(data.index)
# drop index
data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

## Select Data_train

In [ ]:
train_features = [k for k in data.columns if not ('Price' in k)] #or ('Month' in k))]
target = 'Price'
# inputs = [*train_features,target]
inputs = [
    'OldNew' ,  
    'Duration' , 
    'PPDCategoryType',
    # Month
    *[k for k in data.columns if ('Month' in k)],
    # Weights
    # *[k for k in data.columns if ('Weights' in k) and (not ("Transf" in k))],
    # # zscore
    *[k for k in data.columns if ('WeightsTransf' in k)],
    # PropertyType
    *[k for k in data.columns if ('PropertyType' in k)],
    # target
    target
]
data_train = data.loc[:,inputs]

In [ ]:
inputs = [
    # 'OldNew', 
    'Duration', 
    'PPDCategoryType', 
    'TransferMonth_1', 
    'TransferMonth_2', 
    'TransferMonth_3', 
    'TransferMonth_4', 
    'TransferMonth_5', 
    'TransferMonth_6', 
    'TransferMonth_7', 
    'TransferMonth_8', 
    'TransferMonth_9', 
    'TransferMonth_10', 
    'TransferMonth_11', 
    'TransferMonth_12', 
    'CountyWeights', 
    'DistrictWeights', 
    'TownCityWeights', 
    'PostcodeAreaWeights', 
    # 'CountyWeightsTransf', 
    # 'DistrictWeightsTransf', 
    # 'TownCityWeightsTransf',
    # 'PostcodeAreaWeightsTransf',
    'PropertyType_Detached', 
    'PropertyType_Flat', 
    'PropertyType_Other', 
    'PropertyType_SemiDetached', 
    'PropertyType_Terraced', 
    'Price'
]
data_train = data.loc[:,inputs]

In [ ]:
data_train.columns

In [ ]:
print(inputs)
print(data_train.shape)

### Modeling

In [ ]:
from pycaret.regression import *
predictor = setup(
    data = data_train, 
    target = 'Price', 
    normalize=True ,
    session_id=123,
    use_gpu=True
)

In [ ]:
best_model = compare_models()

In [ ]:
print(best_model)

In [ ]:
models()

## plot

In [ ]:
plot_model(best_model, plot = 'residuals')

In [ ]:
plot_model(best_model, plot = 'error')

In [ ]:
plot_model(best_model, plot = 'feature')

## Model tuning

In [ ]:
best_model_tuned = tune_model(best_model,optimize='R2')

## predict tuned model

In [ ]:
best_model_tuned

In [ ]:
[x for x in data_unseen.columns]

In [ ]:
data_unseen_inputs = data_unseen.loc[:,[k for k in inputs if k!='Price']]
data_unseen_output = data_unseen.loc[:,target]

In [ ]:
predictions = predict_model(best_model_tuned, data = data_unseen_inputs)

In [ ]:
predictions.head()

## r2

In [ ]:
from sklearn.metrics import r2_score

r2_score(data_unseen_output, predictions['prediction_label'])

## finalise model

In [ ]:
house_predictor_model = finalize_model(best_model_tuned)

In [ ]:
predictions_finalised = predict_model(house_predictor_model, data = data_unseen_inputs)

In [ ]:
from sklearn.metrics import r2_score

r2_score(data_unseen_output, predictions_finalised['prediction_label'])

## save model

In [ ]:
model_version = 'v2' 

In [ ]:
name_model = model_dir_location + 'house_predictor_model' + '_' + model_version
save_model(house_predictor_model, name_model)

## load model

In [ ]:
model_version = 'v2' 

In [ ]:
name_model = model_dir_location + 'house_predictor_model' + "_" + model_version
inputs = [
    'OldNew' ,  
    'Duration' , 
    'PPDCategoryType',
    # Month
    *[k for k in data.columns if ('Month' in k)],
    # Weights
    *[k for k in data.columns if ('Weights' in k) and (not ("Transf" in k))],
    # # zscore
    *[k for k in data.columns if ('WeightsTransf' in k)],
    # PropertyType
    *[k for k in data.columns if ('PropertyType' in k)],
    # target
    target
]
data_unseen_inputs = data_unseen.loc[:,[k for k in inputs if k!='Price']]

In [ ]:
house_predictor_model = load_model(name_model)

In [ ]:
predictions_mloaded = predict_model(house_predictor_model, data = data_unseen_inputs)

In [ ]:
from sklearn.metrics import r2_score

r2_score(data_unseen_output, predictions_mloaded['prediction_label'])

### Selecting Features
<a name='GDF'></a>
Above, we knew that an $x^2$ term was required. It may not always be obvious which features are required. One could add a variety of potential features to try and find the most useful. For example, what if we had instead tried : $y=w_0x_0 + w_1x_1^2 + w_2x_2^3+b$ ? 

Run the next cells. 